In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
import math
import random

In [ ]:
class PositionalEncoding(nn.Module):
    def __init__(self, d_model: int, dropout: float = 0.1, max_len: int = 5000):
        """
        Arguments:
            d_model: モデルの隠れ層の次元数
            dropout: ドロップアウト率
            max_len: 想定される入力シーケンス最大長
        """

        super().__init__()
        self.dropout = nn.Dropout(p=dropout)

        # Positional Encoding行列[max_len, d_model]の初期化
        pe = torch.zeros(max_len, d_model)

        # 位置情報のベクトル
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)

        # 10000^(2i/d_model)の計算
        # 対数空間で計算してからexpで戻すことで数値安定性を確保
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))

        # 偶数次元にsin、奇数次元にcosを適用
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)

        # バッチ次元を追加してshapeを[1, max_len, d_model]に変形
        pe = pe.unsqueeze(0)

        # モデルのパラメータとして登録（学習されない）
        # state_dictに保存されるが、勾配計算optimizerの対象にはならない
        self.register_buffer('pe', pe)
    
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        Arguments:
            x: Enbeddingされた入力テンソル、shapeは[batch_size, seq_len, d_model]
        
        Returns:
            Positional Encodingが加算されたテンソル、shapeは[batch_size, seq_len, d_model]
        """
        # 入力テンソルの長さに合わせてPositional Encodingをスライスして加算
        x = x + self.pe[:, :x.size(1), :]

        # ドロップアウトを適用して出力
        return self.dropout(x)
    

class MultiHeadAttention(nn.Module):
    def __init__(self, d_model: int, num_heads: int, dropout: float = 0.1):
        """
        Args:
            d_model (int): モデルの隠れ層の次元数
            num_heads (int): ヘッドの数
            dropout (float, optional): ドロップアウト率. Defaults to 0.1.
        """

        super().__init__()

        # d_modelがnum_headsで割り切れることを確認
        assert d_model % num_heads == 0, "d_model must be divisible by num_heads"
        
        self.d_model = d_model
        self.num_heads = num_heads
        self.d_k = d_model // num_heads # 各ヘッドの次元数

        # Q, K, Vの線形変換
        # 実際には全ヘッド分を一度に計算するため、出力次元はd_modelのまま
        self.w_q = nn.Linear(d_model, d_model)
        self.w_k = nn.Linear(d_model, d_model)
        self.w_v = nn.Linear(d_model, d_model)

        self.fc_out = nn.Linear(d_model, d_model)
        self.dropout = nn.Dropout(dropout)
    
    def forward(self, query: torch.Tensor, key: torch.Tensor, value: torch.Tensor, mask: torch.Tensor = None) -> torch.Tensor:
        """

        Args:
            query (torch.Tensor): [batch_size, seq_len, d_model]
            key (torch.Tensor):   [batch_size, seq_len, d_model]
            value (torch.Tensor): [batch_size, seq_len, d_model]
            mask (torch.Tensor, optional): [batch_size, 1, 1, seq_len] または [batch_size, 1, seq_len, seq_len]
                                           (0: マスクなし、1: マスクありなどの定義によるが、ここでは加算マスクを想定)
                                           Defaults to None.

        Returns:
            torch.Tensor: [batch_size, seq_len, d_model]
        """
        batch_size = query.size(0)

        # 1. 線形変換
        # [batch_size, seq_len, d_model] -> [batch_size, seq_len, num_heads]
        Q = self.w_q(query)
        K = self.w_k(key)
        V = self.w_v(value)

        # 2. ヘッドの分割
        # [batch_size, seq_len, num_heads] -> [batch_size, seq_len, num_heads, d_k]
        # その後、計算しやすいようにヘッドの次元を先頭に移動させる(転置) -> [batch_size, num_heads, seq_len, d_k]
        Q = Q.view(batch_size, -1, self.num_heads, self.d_k).transpose(1, 2)
        K = K.view(batch_size, -1, self.num_heads, self.d_k).transpose(1, 2)
        V = V.view(batch_size, -1, self.num_heads, self.d_k).transpose(1, 2)

        # 3. Scaled Dot-Product Attention
        # 3.1. スコアの計算 Q * K^T / sqrt(d_k)
        # Q: [..., seq_len_q, d_k], K^T: [..., d_k, seq_len_k] -> scores: [..., seq_len_q, seq_len_k]
        scores = torch.matmul(Q, K.transpose(-2, -1)) / math.sqrt(self.d_k)

        # 3.2. マスクの適用(optional)
        if mask is not None:
            # ここではマスクが0の場所を非常に小さい値(-1e9)でマスクすると仮定
            # 実装により1と0の定義が異なる場合があるため注意する
            # 非常に小さい値(-1e9)で埋めることで、softmax後にほぼ0になるようにする
            # scores = scores.masked_fill(mask == 0, -1e9)
            scores = scores + mask # 加算マスクの場合 (maskが0の場所に-1e9が入っている想定)
        
        # 3.3. softmax & dropout
        attention_weights = torch.softmax(scores, dim=-1)
        attention_weights = self.dropout(attention_weights)

        # 3.4. Valueとの積
        # attention_weights: [..., seq_len_q, seq_len_k] * V: [..., seq_len_k, d_k] -> [...,seq_lenn_q, d_k]
        output = torch.matmul(attention_weights, V)

        # 4. ヘッドの結合
        # [batch_size, num_beads, seq_len, d_k] -> [batch_size, seq_len, num_heads, d_k]
        output = output.transpose(1, 2).contiguous()

        # [batch_size, seq_len, d_model]に戻す
        output = output.view(batch_size, -1, self.d_model)

        # 5. 線形変換
        output = self.fc_out(output)
        return output


class PositionwiseFeedForward(nn.Module):
    def __init__(self, d_model: int, d_ff: int, dropout: float = 0.1):
        """
        Args:
            d_model (int): モデルの次元数
            d_ff (int): FFNの中間層の次元数
            dropout (float, optional): ドロップアウト率. Defaults to 0.1.
        """
        super().__init__()

        # 一層目 d_model -> d_ff
        self.w_1 = nn.Linear(d_model, d_ff)
        # 二層目 d_ff -> d_model
        self.w_2 = nn.Linear(d_ff, d_model)
        # ドロップアウト
        self.dropout = nn.Dropout(dropout)
        # 活性化関数 ReLU
        # 元論文ではReLUが使われているが、近年のLLMではGELUがよく使われている
        self.activation = nn.ReLU()
    
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        Args:
            x (torch.Tensor): [batch_size, seq_len, d_model]

        Returns:
            torch.Tensor: [batch_size, seq_len, d_model]
        """
        # Linear -> ReLU -> Dropout -> Linear
        # x: [batch_size, seq_len, d_model] -> [batch_size, seq_len, d_ff]
        hidden = self.activation(self.w_1(x))
        hidden = self.dropout(hidden)

        # [batch_size, seq_len, d_ff] -> [batch_size, seq_len, d_model]
        output = self.w_2(hidden)
        return output

In [ ]:
class EncoderLayer(nn.Module):
    def __init__(self, d_model: int, num_heads: int, d_ff: int, dropout: float = 0.1):
        super().__init__()

        # 1. Self-Attention layer
        self.self_attn = MultiHeadAttention(d_model, num_heads, dropout)

        # 2. Feed-Forward Network layer
        self.ffn = PositionwiseFeedForward(d_model, d_ff, dropout)

        # 3. Layer Normalization & Dropout layers
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)
    
    def forward(self, x: torch.Tensor, mask: torch.Tensor = None) -> torch.Tensor:
        """

        Args:
            x (torch.Tensor): [batch_size, seq_len, d_model]
            mask (torch.Tensor, optional): Padding Maskなど. Defaults to None.

        Returns:
            torch.Tensor: [batch_size, seq_len, d_model]
        """

        # 1. Sublayer 1: Self-Attention
        # Residual Connection: x + Sublayer(x)
        # Post-LN: Norm(x + Sublayer(x))
        # Attentionの入出力は同じshape
        attn_output = self.self_attn(x, x, x, mask)
        x = self.norm1(x + self.dropout(attn_output))

        # 2. Sublayer 2: Feed-Forward Network
        ffn_output = self.ffn(x)
        x = self.norm2(x + self.dropout(ffn_output))

        return x


class DecoderLayer(nn.Module):
    def __init__(self, d_model: int, num_heads: int, d_ff: int, dropout: float = 0.1):
        super().__init__()

        # 1. Masked Self-Attention layer
        self.self_attn = MultiHeadAttention(d_model, num_heads, dropout)

        # 2. Cross-Attention layer (Source-Target Attention)
        self.cross_attn = MultiHeadAttention(d_model, num_heads, dropout)

        # 3. Feed-Forward Network layer
        self.ffn = PositionwiseFeedForward(d_model, d_ff, dropout)

        # 4. Normalization & Dropout layers
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.norm3 = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)
    
    def forward(self, x: torch.Tensor, memory: torch.Tensor, src_mask: torch.Tensor, tgt_mask: torch.Tensor) -> torch.Tensor:
        """

        Args:
            x (torch.Tensor): Decoderへの入力テンソル、shapeは[batch_size, tgt_len, d_model]
            memory (torch.Tensor): Encoderの出力テンソル、shapeは[batch_size, src_len, d_model]
            src_mask (torch.Tensor): Memoryに対するマスク(Padding Mask)
            tgt_mask (torch.Tensor): Self-Attention用のマスク(Look-Ahead Mask + Padding Mask)

        Returns:
            torch.Tensor: Decoderの出力テンソル、shapeは[batch_size, tgt_len, d_model]
        """

        # 1. Sublayer 1: Masked Self-Attention
        # 未来の単語を見ないように tgt_mask を適用
        attn_output = self.self_attn(x, x, x, tgt_mask)
        x = self.norm1(x + self.dropout(attn_output))

        # 2. Sublayer 2: Cross-Attention
        # Query = x(Decoderの出力), Key = Value = memory(Encoderの出力)
        # Encoder側のパディングを見ないように src_mask を適用
        attn_output = self.cross_attn(x, memory, memory, src_mask)
        x = self.norm2(x + self.dropout(attn_output))

        # 3. Sublayer 3: Feed-Forward Network
        ffn_output = self.ffn(x)
        x = self.norm3(x + self.dropout(ffn_output))

        return x


class Encoder(nn.Module):
    def __init__(self, vocab_size: int, d_model: int, num_layers: int, num_heads: int, d_ff: int, max_len: int, dropout: float = 0.1):
        super().__init__()

        self.embedding = nn.Embedding(vocab_size, d_model)
        self.pos_encoding = PositionalEncoding(d_model, dropout, max_len)

        # EncoderLayerをnum_layers個積み重ねる
        self.layers = nn.ModuleList([
            EncoderLayer(d_model, num_heads, d_ff, dropout) for _ in range(num_layers)
        ])

        self.dropout = nn.Dropout(dropout)
        self.d_model = d_model

    def forward(self, x: torch.Tensor, mask: torch.Tensor) -> torch.Tensor:
        # 1. Embedding & Positional Encoding
        # 論文通り sqrt(d_model)でスケーリング
        x = self.embedding(x) * math.sqrt(self.d_model)
        x = self.pos_encoding(x)

        # 2. Apply all layers
        for layer in self.layers:
            x = layer(x, mask)
        
        return x


class Decoder(nn.Module):
    def __init__(self, vocab_size: int, d_model: int, num_layers: int, num_heads: int, d_ff: int, max_len: int, dropout: float = 0.1):
        super().__init__()

        self.embedding = nn.Embedding(vocab_size, d_model)
        self. pos_encoding = PositionalEncoding(d_model, dropout, max_len)

        # DecoderLayerをnum_layers個積み重ねる
        self.layers = nn.ModuleList([
            DecoderLayer(d_model, num_heads, d_ff, dropout) for _ in range(num_layers)
        ])

        self.dropout = nn.Dropout(dropout)
        self.d_model = d_model

    def forward(self, x: torch.Tensor, memory: torch.Tensor, src_mask: torch.Tensor, tgt_mask: torch.Tensor) -> torch.Tensor:
        # 1. Embedding & Positional Encoding
        # 論文通り sqrt(d_model)でスケーリング
        x = self.embedding(x) * math.sqrt(self.d_model)
        x = self.pos_encoding(x)

        # 2. Apply all layers
        for layer in self.layers:
            x = layer(x, memory, src_mask, tgt_mask)
        
        return x


class Transformer(nn.Module):
    def __init__(self, src_vocab_size: int, tgt_vocab_size: int, d_model: int = 512, num_layers: int = 6, num_heads: int = 8, d_ff: int = 2048, max_len: int = 5000, dropout: float = 0.1):
        super().__init__()

        self.encoder = Encoder(src_vocab_size, d_model, num_layers, num_heads, d_ff, max_len, dropout)
        self.decoder = Decoder(tgt_vocab_size, d_model, num_layers, num_heads, d_ff, max_len, dropout)

        # 最終出力層の線形変換(Linear Projection)
        self.fc_out = nn.Linear(d_model, tgt_vocab_size)

    def forward(self, src: torch.Tensor, tgt: torch.Tensor, src_mask: torch.Tensor = None, tgt_mask: torch.Tensor = None) -> torch.Tensor:
        """

        Args:
            src (torch.Tensor): [batch, src_len] Encoderへの入力単語ID列
            tgt (torch.Tensor): [batch, tgt_len] Decoderへの入力単語ID列
            src_mask (torch.Tensor, optional): Encoder用のマスク. Defaults to None.
            tgt_mask (torch.Tensor, optional): Decoder用のマスク. Defaults to None.

        Returns:
            torch.Tensor: [batch, tgt_len, tgt_vocab_size] 出力単語の確率分布
        """

        # 1. Encode
        # memory: [batch, src_len, d_model]
        memory = self.encoder(src, src_mask)

        # 2. Decode
        # decoder_output: [batch, tgt_len, d_model]
        decoder_output = self.decoder(tgt, memory, src_mask, tgt_mask)

        # 3. Final linear layer
        # logits: [batch, tgt_len, tgt_vocab_size]
        logits = self.fc_out(decoder_output)

        return logits
    
    def encode(self, src: torch.Tensor, src_mask: torch.Tensor = None) -> torch.Tensor:
        """推論時にEncoderのみを動かすためのヘルパー"""
        return self.encoder(src, src_mask)
    
    def decode(self, tgt: torch.Tensor, memory: torch.Tensor, src_mask: torch.Tensor = None, tgt_mask: torch.Tensor = None) -> torch.Tensor:
        """推論時にDecoderのみを動かすためのヘルパー"""
        return self.decoder(tgt, memory, src_mask, tgt_mask)

In [ ]:
def create_padding_mask(seq: torch.Tensor, pad_idx: int = 0) -> torch.Tensor:
    """パディングマスクを作成する関数。<pad>の部分を1e-9、それ以外を0にする。

    Args:
        seq (torch.Tensor): [batch_size, seq_len]の形状を持つテンソル。入力単語のID列
        pad_idx (int, optional): パディングを表すID。 Defaults to 0.

    Returns:
        torch.Tensor: [batch_size, 1, 1, seq_len] Mult-head Attentionのスコアに加算するためのマスク
    """

    # seq == pad_idx の部分はTrue、それ以外はFalse
    mask = (seq == pad_idx)

    # shapeを [batch_size, 1, 1, seq_len] に変形
    # Trueを1e-9、Falseを0に変換
    # float型に変換しないと加算時にエラーになる
    return mask.unsqueeze(1).unsqueeze(2).float() * -1e9


def create_look_ahead_mask(seq_len: int) -> torch.Tensor:
    """未来の単語を見えなくするための上三角マスクを作成する関数

    Args:
        seq_len (int): シーケンス長

    Returns:
        torch.Tensor: [1, 1, seq_len, seq_len] 対角成分より上が1e-9、それ以外が0の行列
    """
    # torch.triuで上三角行列を取り出す (diagonal=1で対角線のひとつ上から)
    mask = torch.triu(torch.ones(seq_len, seq_len), diagonal=1)

    # マスク部分を1e-9、それ以外を0に変換
    return mask.unsqueeze(0).unsqueeze(0).float() * -1e9


def create_masks(src: torch.Tensor, tgt: torch.Tensor, pad_idx: int = 0):
    """EncoderとDecoderに必要なすべてのマスクを一括生成するヘルパー関数

    Args:
        src (torch.Tensor): [batch_size, src_len]
        tgt (torch.Tensor): [batch_size, tgt_len]
        pad_idx (int, optional): パディングID。 Defaults to 0.
    """
    # 1. Encoder用のマスク(Padding Maskのみ)
    src_mask = create_padding_mask(src, pad_idx)

    # 2. Decoder用のマスク(Padding Mask + Look-ahead Mask)
    # 2.1. TargetのPadding Mask [batch_size, 1, 1, tgt_len]
    tgt_pad_mask = create_padding_mask(tgt, pad_idx)

    # 2.2. Look-ahead Mask [1, 1, tgt_len, tgt_len]
    tgt_len = tgt.size(1)
    look_ahead_mask = create_look_ahead_mask(tgt_len).to(tgt.device)

    # 2.3. Padding MaskとLook-ahead Maskを結合 (どちらかが-1e9なら-1e9になるように加算またはmaxを取る)
    # ここで単純に和を取ると-2e9になる箇所ができるが、Softmaxにおいては十分小さいので計算には影響しない
    # 論理和(OR)的にマスクしたいので、最小値を取る実装もよくある(torch.min)
    tgt_mask = torch.min(tgt_pad_mask, look_ahead_mask)

    return src_mask, tgt_mask


class NoamScheduler:
    def __init__(self, optimizer: torch.optim.Optimizer, d_model: int, warmup_steps: int = 4000):
        """Transformer用学習率スケジューラ (Noam Scheduler)

        Args:
            optimizer (torch.optim.Optimizer): PytorchのOptimizer
            d_model (int): モデルの次元数
            warmup_steps (int, optional): ウォームアップステップ数。 Defaults to 4000.
        """
        self.optimizer = optimizer
        self.d_model = d_model
        self.warmup_steps = warmup_steps
        self.current_step = 0

    def step(self):
        """1ステップ進め、学習率を更新し、optimizer.step()を実行する
        """
        self.current_step += 1
        lr = self.get_lr()

        # Optimizerの学習率を更新
        for param_group in self.optimizer.param_groups:
            param_group['lr'] = lr
        
        self.optimizer.step()
    
    def get_lr(self) -> float:
        """現在のステップ数に基づき学習率を計算

        Returns:
            float: 学習率
        """
        step = self.current_step
        return (self.d_model ** -0.5) * min(step ** -0.5, step * (self.warmup_steps ** -1.5))
    
    def zero_grad(self):
        """optimizerの勾配を初期化(ゼロにする)
        """
        self.optimizer.zero_grad()

In [ ]:
def train_epoch(model: nn.Module, data_loader: list, optimizer: torch.optim.Optimizer, criterion: nn.Module, device: torch.device) -> float:
    """1エポック分の学習を行う関数

    Args:
        model (nn.Module): transformerモデル
        data_loader (list): バッチデータのリスト
        optimizer (torch.optim.Optimizer): オプティマイザ (NoamSchedulerでラップされている想定)
        criterion (nn.Module): 損失関数 (LabelSmoothingCrossEntropy)
        device (torch.device): 計算デバイス(CPU or GPU)

    Returns:
        float: 1エポック分の平均損失
    """
    model.train() # 学習モード(Dropout有効)
    total_loss = 0.0
    
    for i, batch in enumerate(data_loader):
        src, tgt = batch
        src = src.to(device)
        tgt = tgt.to(device)

        # targetの処理
        # device_input: <sos>, w1, w2, ... (最後を含まない)
        # target_label: w1, w2, ..., <eos> (最初を含まない)
        tgt_input = tgt[:, :-1]
        tgt_label = tgt[:, 1:]

        # マスクの作成
        # tgt_inputを使ってマスクを作る点に注意
        src_mask, tgt_mask = create_masks(src, tgt_input, pad_idx=0)

        # Forward pass
        # output: [batch_size, tgt_len, vocab_size]
        output = model(src, tgt_input, src_mask, tgt_mask)

        # Loss計算
        # CrossEntropyLoss は [N, C] の入力を期待するため、出力とラベルを変形
        # output: [batch_size * tgt_len, vocab_size]
        # label: [batch_size * tgt_len]
        loss = criterion(output.contiguous().view(-1, output.size(-1)), tgt_label.contiguous().view(-1))

        # Backward & Update
        optimizer.zero_grad()
        loss.backward()
        optimizer.step() # NoamScheduler.step()内でoptimizer.step()が呼ばれる

        total_loss += loss.item()
    
    return total_loss / len(data_loader)

In [ ]:
def greedy_decode(model: nn.Module, src: torch.Tensor, src_mask: torch.Tensor, max_len: int, start_symbol: int, end_symbol: int, device: torch.device) -> torch.Tensor:
    """Greedy Decodingによる推論を行う関数

    Args:
        model (nn.Module): 学習済みTransformerモデル
        src (torch.Tensor): [1, src_len] エンコードする入力シーケンス
        src_mask (torch.Tensor): [1, 1, 1, src_len] エンコード入力に対するマスク
        max_len (int): 最大生成長
        start_symbol (int): 開始トークン<sos>のID
        end_symbol (int): 終了トークン<eos>のID
        device (torch.device): 計算デバイス(CPU or GPU)

    Returns:
        torch.Tensor: [1, gen_len] 生成されたシーケンスのトークンID列
    """
    model.eval() # 推論モード(Dropout無効)

    # 1. Encode(一度だけ実行)
    # memory: [1, seq_len, d_model]
    memory = model.encode(src, src_mask)

    # 2. Decode (ループ処理)
    # 生成シーケンスの初期化 (開始トークン <sos> のみ)
    ys = torch.ones(1, 1).fill_(start_symbol).type_as(src.data)

    for i in range(max_len):
        # 現在のysに対するマスクを作成
        tgt_mask = create_look_ahead_mask(ys.size(1)).to(device)

        # Decoderを通す
        # out: [1, current_seq_len, d_model]
        out = model.decode(ys, memory, src_mask, tgt_mask)

        # 最後の時刻の出力を使って次の単語を予測
        # prob: [1, vocab_size]
        prob = model.fc_out(out[:, -1])

        # 確率最大の単語IDを取得 (Greedy選択)
        _, next_word = torch.max(prob, dim=1)
        next_word = next_word.item()

        # 生成された単語をリストに追加
        ys = torch.cat([ys, torch.ones(1, 1).type_as(src.data).fill_(next_word)], dim=1)

        # 終了トークン<eos>が生成されたら終了
        if next_word == end_symbol:
            break
    
    return ys

In [ ]:
class CopyTaskDataset(Dataset):
    def __init__(self, num_samples: int, max_len: int, vocab_size: int):
        """ランダムな文字列のペア(src, tgt)を生成・保持するデータセットクラス
        Args:
            num_samples (int): 生成するサンプル数
            max_len (int): 各サンプルの最大長
            vocab_size (int): 単語IDの語彙数
        """
        self.num_samples = num_samples
        self.max_len = max_len
        self.vocab_size = vocab_size
        self.data = self._generate_data()

    def _generate_data(self) -> list:
        data = []

        # <sos>と<eos>のIDを定義
        start_symbol = self.vocab_size
        end_symbol = self.vocab_size + 1
        for _ in range(self.num_samples):
            # 1〜vocab_size-1 のランダムな数列(0はpad, vocab_size+2はstart/end token用に空けておく
            seq_len = random.randint(1, self.max_len)

            # ランダムな長さの数字列
            seq = torch.randint(1, self.vocab_size, (seq_len,))

            # Padding処理
            # src: [seq_len] -> [max_len] に0埋め
            src = torch.zeros(self.max_len + 2, dtype=torch.long)
            src[:seq_len] = seq

            # tgt: [<sos>, ..., <eos>] -> [max_len + 2] に0埋め
            tgt = torch.zeros(self.max_len + 2, dtype=torch.long)
            tgt[0] = start_symbol
            tgt[1:seq_len + 1] = seq
            tgt[seq_len + 1] = end_symbol
            # 残りは0でパディングされたまま

            data.append((src, tgt))
        return data
    
    def __len__(self) -> int:
        return self.num_samples
    
    def __getitem__(self, idx: int) -> tuple:
        # インデックスに対応する(src, tgt)ペアを返す
        return self.data[idx]

def collate_fn(batch) -> tuple:
    """DataLoaderがミニバッチを作成する際に呼ばれるコールバック関数

    Args:
        batch (list): (src, tgt)ペアのリスト [(src1, tgt1), (src2, tgt2), ...]

    Returns:
        tuple: パディングされたsrcとtgtのテンソル
    """
    # バッチ内のsrcとtgtのペアを分離
    src_list, tgt_list = zip(*batch)

    # リストをTensorに変換してスタック
    # [batch_size, seq_len]になる
    src_batch = torch.stack(src_list)
    tgt_batch = torch.stack(tgt_list)

    return src_batch, tgt_batch

In [ ]:
# ハイパーパラメータ設定
SRC_VOCAB = 100
TGT_VOCAB = 100 + 2  # +2 for <sos> and <eos>
D_MODEL = 128
D_FF = 512
NUM_LAYERS = 2
NUM_HEADS = 4
DROPOUT = 0.1
MAX_LEN = 20

# 学習設定
BATCH_SIZE = 32
EPOCHS = 100
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {DEVICE}')
# 1. モデルの構築
model = Transformer(
    src_vocab_size=SRC_VOCAB, 
    tgt_vocab_size=TGT_VOCAB, 
    d_model=D_MODEL, 
    num_layers=NUM_LAYERS, 
    num_heads=NUM_HEADS, 
    d_ff=D_FF, 
    max_len=MAX_LEN + 2, # +2 for <sos> and <eos>
    dropout=DROPOUT,
).to(DEVICE)

# 重みの初期化 Xavier Initialization
for p in model.parameters():
    if p.dim() > 1:
        nn.init.xavier_uniform_(p)

# 2. オプティマイザとスケジューラ
optimizer = torch.optim.Adam(model.parameters(), lr=0, betas=(0.9, 0.98), eps=1e-9)
scheduler = NoamScheduler(optimizer=optimizer, d_model=D_MODEL, warmup_steps=1000)

# 3. 損失関数 Label Smoothing Cross Entropy
# pad_idx=0を無視するように設定
criterion = nn.CrossEntropyLoss(ignore_index=0, label_smoothing=0.0)

# 4. ダミーデータの生成とデータローダの作成
dataset = CopyTaskDataset(num_samples=1000, max_len=MAX_LEN, vocab_size=SRC_VOCAB)
data_loader = DataLoader(
    dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
    collate_fn=collate_fn, # カスタムコールバック関数(バッチ化の処理)を指定
    drop_last=True # 最後の端数バッチが小さい場合に破棄(サイズを固定したい場合)
)

In [ ]:
# 5. 学習ループ実行
print("Start training...")
for epoch in range(EPOCHS):
    loss = train_epoch(model, data_loader, scheduler, criterion, DEVICE)
    print(f"Epoch: {epoch+1: 02} | Loss: {loss:.4f} | LR: {scheduler.get_lr():.6f}")

In [ ]:
# 6. 推論の確認
print("Start inference...")
model.eval()
test_src = torch.tensor([[1, 2, 3, 4, 5]]).to(DEVICE)  # [1, src_len]
src_mask = create_padding_mask(test_src, pad_idx=0).to(DEVICE)

# 推論実行
# start_symbol=100, end_symbol=101を指定
generated = greedy_decode(model, test_src, src_mask, max_len=10, start_symbol=100, end_symbol=101, device=DEVICE)
print(f"Input Sequence: {test_src.cpu().numpy()}")
print(f"Generated Sequence: {generated.cpu().numpy()}")